In [100]:
import pickle
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import numpy
nltk.download('punkt')



def cluster_score(cluster):
    sig_words = len(cluster)
    total_words = cluster[-1] - cluster[0] + 1
    return sig_words ** 2 / total_words

def score_sentences(sentences, important_words, THRESH = 5):
    scores = []
    
    for sent in map(word_tokenize, sentences):
        word_idx = []
        
        for word in important_words:
            if word in sent:
                word_idx.append(sent.index(word))
        word_idx.sort()
        if len(word_idx) > 0:
            clusters = []
            current_cluster = [word_idx[0]]
            for idx in word_idx[1:]:
                if idx - word_idx[-1] < THRESH:
                    current_cluster.append(idx)
                else:
                    clusters.append(current_cluster)
                    current_cluster = [idx]
            clusters.append(current_cluster)
            scores.append(max(map(cluster_score, clusters)))
        else:
            scores.append(0)
    return scores

def summarize(sentences, important_words, CTHRESH = 5, NUM_SENTENCES = 5):
    scores = score_sentences(sentences, important_words, CTHRESH)
    avg = numpy.mean(scores)
    std_dev = numpy.std(scores)
    score_threshold = avg + 0.5 * std_dev
    mean_scored = [t[0] for t in enumerate(scores) if t[1] > score_threshold]
    
    sorted_scores = sorted(enumerate(scores), key=lambda x: x[1], reverse = True)[:NUM_SENTENCES]
    sorted_indexes = sorted([s[0] for s in sorted_scores])
    
    return (' '.join([sentences[i] for i in sorted_indexes]))



#https://www.msn.com/en-us/news/world/putin-russia-knows-mueller-probe-gave-birth-to-a-mouse/ar-BBVLtjG?ocid=spartanntp
#https://www.msn.com/en-us/health/health-news/a-new-cancer-vaccine-uses-the-immune-system-to-take-out-tumors/ar-BBVLDdK?ocid=spartanntp
r = requests.get('https://www.msn.com/en-us/news/world/putin-russia-knows-mueller-probe-gave-birth-to-a-mouse/ar-BBVLtjG?ocid=spartanntp')

text = 'Short stories have deep roots and the power of short fiction has been recognised in modern society for hundreds of years. The short form is, conceivably, more natural to us than longer forms. We are drawn to short stories as the well-told story, and as William Boyd, the award-winning British author and short story writer has said:'

soup = BeautifulSoup(r.text, 'html.parser')

stopWords = set(stopwords.words("english"))
#rint(stopWords)
#print(r.text)
#print(soup.findAll('p'))
#print(soup.prettify())
paras = soup.findAll('p', string=True)

#print(paras)
paragraphs = []
for para in paras:
    paragraphs.append(para.text)

#print(paragraphs)
#getting a list of sentences from an article
exSentences = sent_tokenize(text)
#print(exSentences)
sentences = []
sentences = [sent_tokenize(sent) for sent in paragraphs]
#print(sentences)
#for i in range (len(paragraphs)):
#    sentences.append(sent_tokenize(paragraphs[i]))
#print(sentences)
with open('sentences.pkl', 'wb') as f:
    pickle.dump(sentences, f)
with open('sentences.pkl', 'rb') as f:
    pickled_sents = pickle.load(f)
#print(pickled_sents)

listsents = []
for a in range(len(sentences)):
    for sent in sentences[a]:
        newsents = re.sub('[-_\/]', '', sent)
        listsents.append(newsents)
        
print(listsents)


#tokens = [word_tokenize(s) for s in sentences]
#print(tokens)
#getting a list of words
#words = list()
#for sent in sentences:
#    words.extend(map(lambda x: x.lower(), word_tokenize(sent)))
#print(words)
"""lettersList = []
for a in range(len(sentences)):
    for sent in sentences[a]:
        print(sent)
        for i in range(len(sent)):
            lettersList.append(sent[i])
print(lettersList)

for sent in sentences:
    newsenteneces = re.sub('[^a-zA-Z]', ' ', sent)
print(newsentences)
"""
newList = []

for a in range(len(sentences)):
    for sent in sentences[a]:
        newsent = re.sub('[^a-zA-Z]', ' ', sent)
        #print(newsent)
        newList.append(newsent)
#print(newList)
tokens = [word_tokenize(s) for s in newList]
#print(tokens)
#print([s.strip(';', ',','.', '?', "'", '"', ':', '/', '_', '-',) for s in sentences])

#print([s.replace("\"" ,'') for s in sentences])

#for sent in sentences:
    #newLine = re.sub('/\,.?:;""-_''', '', sent)
    #print(newLine)
article_words = []
for toke in tokens:
    article_words += toke
        
#print(article_words)

for punct in "_-,.'?:;'""":
    stopWords.add(punct)
freqDist = nltk.FreqDist(article_words)

num_sents = 5

sorted_words = sorted(freqDist.items(), key=lambda x: x[1], reverse=True)

most_common_words = [word[0] for word in sorted_words
                    if word[0] not in stopWords][:num_sents]

print(most_common_words)

auto_summary_txt = summarize(listsents, most_common_words, 5, 5)
print(auto_summary_txt)

#print(words)
phrase = 'Short stories have deep roots'
auto_summary = summarize(exSentences, phrase, 5, 2)

#print(auto_summary)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jzech\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jzech\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jzech\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['ST. PETERSBURG, Russia  — Russian President Vladimir Putin on Tuesday mocked U.S. special counsel Robert Mueller\'s investigation of Kremlin interference in the 2016 presidential election, saying "a mountain gave birth to a mouse."', "In his first comments since Mueller finished his probe, Putin sought to cast the 22month investigation as a failure and disregarded the special counsel's exposure of a Russian operation to put Donald Trump in the White House.", '"It was clear for us from the start that it would end like this," the Russian leader said as the Trump administration and Congress sparred over making Mueller\'s stillconfidential investigation report public.', 'Attorney General William Barr wrote in a summary of Mueller\'s report that the special counsel found no evidence the Trump campaign "conspired or coordinated" with the Russian government to influence the election.', 'However, Mueller uncovered evidence of a Kremlin operation to interfere with the 2016 vote.', "He charged